In [1]:
from haystack import Pipeline
from haystack.components.builders import PromptBuilder
from haystack.components.generators import HuggingFaceLocalGenerator

# 添加日誌
import logging

# 設置日誌輸出級別
logging.basicConfig(level=logging.INFO)

# 定義模板，將用戶輸入的主題插入其中
# 英文
template = """
Please create a summary about the following topic:
{{ topic }}
"""

# 建立 PromptBuilder，使用定義的模板
builder = PromptBuilder(template=template)
logging.info("PromptBuilder 已創建。")

# 建立 HuggingFaceLocalGenerator，指定使用的模型和生成的參數
logging.info("開始加載模型，這可能需要一些時間。")
llm = HuggingFaceLocalGenerator(
    model="google/flan-t5-large",
    task="text2text-generation",
    generation_kwargs={"max_new_tokens": 300, "temperature": 0.7},
)
logging.info("HuggingFaceLocalGenerator 已創建並初始化。")

# 建立管道
pipeline = Pipeline()
# 添加組件到管道
pipeline.add_component(name="builder", instance=builder)
pipeline.add_component(name="llm", instance=llm)

# 連接管道中的組件
pipeline.connect("builder", "llm")
logging.info("管道組件已連接。")

# 定義用戶輸入的主題
topic = "Climate change"

# 輸出生成的摘要
logging.info("正在運行管道...")
# 運行管道，生成關於主題的摘要
result = pipeline.run(data={"builder": {"topic": topic}})
summary = result["llm"]["replies"][0]
print(f"查看整個 result：{result}")
# 增加檢查生成的結果
if summary.strip() == "":
    logging.warning("生成的摘要為空。")
else:
    print("生成的摘要：", summary)

/Users/samhsiao/Documents/PythonVenv/envHaystack/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:root:PromptBuilder 已創建。
INFO:root:開始加載模型，這可能需要一些時間。
INFO:root:HuggingFaceLocalGenerator 已創建並初始化。
INFO:root:管道組件已連接。
INFO:root:正在運行管道...
INFO:haystack.core.pipeline.base:Warming up component llm...
/Users/samhsiao/Documents/PythonVenv/envHaystack/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
INFO:haystack.core.pipeline.pipeline:Running component builder
INFO:haystack.core.pipeline.pipeline:Running component llm
/Users/samhsiao/Documents/PythonVenv/envHaystack/lib/python3.10/site-pa

查看整個 result：{'llm': {'replies': ['Climate change is a major threat to the planet.']}}
生成的摘要： Climate change is a major threat to the planet.


In [2]:
# 將管道序列化為 YAML
yaml_pipeline = pipeline.dumps()
print("管道的 YAML 表示：\n", yaml_pipeline)

管道的 YAML 表示：
 components:
  builder:
    init_parameters:
      required_variables: null
      template: '

        Please create a summary about the following topic:

        {{ topic }}

        '
      variables: null
    type: haystack.components.builders.prompt_builder.PromptBuilder
  llm:
    init_parameters:
      generation_kwargs:
        max_new_tokens: 300
        temperature: 0.7
      huggingface_pipeline_kwargs:
        device: mps
        model: google/flan-t5-large
        task: text2text-generation
      stop_words: null
      streaming_callback: null
      token:
        env_vars:
        - HF_API_TOKEN
        strict: false
        type: env_var
    type: haystack.components.generators.hugging_face_local.HuggingFaceLocalGenerator
connections:
- receiver: llm.prompt
  sender: builder.prompt
max_loops_allowed: 100
metadata: {}



In [22]:
yaml_pipeline = """
components:
  builder:
    init_parameters:
      template: "\nPlease translate the following to French: \n{{ sentence }}\n"
    type: haystack.components.builders.prompt_builder.PromptBuilder
  llm:
    init_parameters:
      generation_kwargs:
        max_new_tokens: 150
      huggingface_pipeline_kwargs:
        device: cpu
        model: google/flan-t5-large
        task: text2text-generation
        token: null
      stop_words: null
    type: haystack.components.generators.hugging_face_local.HuggingFaceLocalGenerator
connections:
- receiver: llm.prompt
  sender: builder.prompt
max_loops_allowed: 100
metadata: {}
"""

In [23]:
from haystack import Pipeline
from haystack.components.builders import PromptBuilder
from haystack.components.generators import HuggingFaceLocalGenerator

# 將 YAML 管道加載為 Python 管道
logging.info("正在從 YAML 加載新的管道...")
new_pipeline = Pipeline.loads(yaml_pipeline)

# 運行新管道，將句子翻譯為法語
logging.info("正在運行新的管道...")
new_result = new_pipeline.run(
    data={"builder": {"sentence": "I love capybaras"}})
translation = new_result["llm"]["replies"][0]
print("翻譯結果：", translation)

INFO:root:正在從 YAML 加載新的管道...


AttributeError: 'NoneType' object has no attribute 'items'